<a href="https://colab.research.google.com/github/gdeotale/E4P2/blob/master/Session9/Neural_word_embeddings_practice_notpart_of_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [23]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# test_sentence
# raw_text

['We',
 'are',
 'about',
 'to',
 'study',
 'the',
 'idea',
 'of',
 'a',
 'computational',
 'process.',
 'Computational',
 'processes',
 'are',
 'abstract',
 'beings',
 'that',
 'inhabit',
 'computers.',
 'As',
 'they',
 'evolve,',
 'processes',
 'manipulate',
 'other',
 'abstract',
 'things',
 'called',
 'data.',
 'The',
 'evolution',
 'of',
 'a',
 'process',
 'is',
 'directed',
 'by',
 'a',
 'pattern',
 'of',
 'rules',
 'called',
 'a',
 'program.',
 'People',
 'create',
 'programs',
 'to',
 'direct',
 'processes.',
 'In',
 'effect,',
 'we',
 'conjure',
 'the',
 'spirits',
 'of',
 'the',
 'computer',
 'with',
 'our',
 'spells.']

In [24]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

# trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
#             for i in range(len(test_sentence) - 2)]
# # print the first 3, just so you can see what they look like
# print(trigrams[:3])

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [30]:
# vocab = set(test_sentence)

vocab = set(raw_text)
vocab_size = len(vocab)
print(vocab_size)
word_to_ix = {word: i for i, word in enumerate(vocab)}
# word_to_ix

49


In [41]:
# class NGramLanguageModeler(nn.Module):

#     def __init__(self, vocab_size, embedding_dim, context_size):
#         super(NGramLanguageModeler, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.linear1 = nn.Linear(context_size * embedding_dim, 128)
#         self.linear2 = nn.Linear(128, vocab_size)

#     def forward(self, inputs):
#         # print(inputs.shape)
#         embeds = self.embeddings(inputs).view((1, -1))
#         # print(embeds.shape)
#         out = F.relu(self.linear1(embeds))
#         out = self.linear2(out)
#         log_probs = F.log_softmax(out, dim=1)
#         return log_probs

class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
      super(CBOW, self).__init__()
      self.embeddings = nn.Embedding(vocab_size, embedding_dim)
      self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)
      self.linear2 = nn.Linear(128, vocab_size)
        # pass

    def forward(self, inputs):
      embeds = self.embeddings(inputs).view((1, -1))
      # print(embeds.shape)
      out = F.relu(self.linear1(embeds))
      out = self.linear2(out)
      log_probs = F.log_softmax(out, dim=1)
      return log_probs
        # pass


# !pip install torchsummary
!pip install torch-summary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
model = CBOW(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE)
summary(model, (2,), dtypes=[torch.long],)

Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 2, 10]               490
├─Linear: 1-2                            [-1, 128]                 5,248
├─Linear: 1-3                            [-1, 49]                  6,321
Total params: 12,059
Trainable params: 12,059
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05


Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 2, 10]               490
├─Linear: 1-2                            [-1, 128]                 5,248
├─Linear: 1-3                            [-1, 49]                  6,321
Total params: 12,059
Trainable params: 12,059
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05

In [42]:
?summary 

In [43]:
model

CBOW(
  (embeddings): Embedding(49, 10)
  (linear1): Linear(in_features=40, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=49, bias=True)
)

In [44]:
losses = []
loss_function = nn.NLLLoss()
# model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
model = CBOW(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [45]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

for epoch in range(200):
    total_loss = 0
    # for context, target in trigrams:
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        context_idxs = make_context_vector(context, word_to_ix)
        # context_idxs = context_idxs.to(device)
        # print(context_idxs.shape)
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)


In [46]:
print(losses)  

[229.22864627838135, 227.83819675445557, 226.45723843574524, 225.08503818511963, 223.72229075431824, 222.3670620918274, 221.0209686756134, 219.68244695663452, 218.35019326210022, 217.023530960083, 215.70147800445557, 214.38367748260498, 213.07029628753662, 211.7609121799469, 210.4551877975464, 209.15214252471924, 207.85123753547668, 206.55268740653992, 205.2561662197113, 203.9607560634613, 202.66791224479675, 201.37641429901123, 200.08639311790466, 198.7975790500641, 197.50996279716492, 196.22253155708313, 194.93501567840576, 193.64668464660645, 192.35688638687134, 191.06495761871338, 189.76912140846252, 188.473379611969, 187.17629647254944, 185.87804174423218, 184.57839179039001, 183.2786670923233, 181.97783386707306, 180.6752896308899, 179.37168955802917, 178.066836476326, 176.75898158550262, 175.44882345199585, 174.13671624660492, 172.82328927516937, 171.50844359397888, 170.1907823085785, 168.8705039024353, 167.55089402198792, 166.22871053218842, 164.9056135416031, 163.5813649892807

In [47]:
make_context_vector(data[0][0], word_to_ix)

tensor([38,  5, 47, 21])